In [ ]:
import functools
import math

In [ ]:
def removeLetters(boxes):
    n = int(math.pow(len(boxes),1/2))
    for i,box in enumerate(boxes):
      a = int(box[0][1]) # Solamente existe Z9 2 valores, A,B,C..Z y 1,2...9
      b = box[1]
      if a!=1:
        b = b.replace('B','')
      if (i+1)>len(boxes)-n:
        continue
      b = b.replace('R','')
      boxes[i]=(boxes[i][0],b)

    return boxes

def flatten_boxes(boxes):
    return removeLetters(boxes)

    #return functools.reduce(
        #lambda m, box: m + list(map(lambda side: f'{box[0]} {side}',list(box[1])))
    #, boxes, [])
flatten_boxes([('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')])

[('A1', 'LTB'), ('A2', 'LT'), ('B1', 'LTRB'), ('B2', 'LTRB')]

In [ ]:
def remove_move(boxes, move):
    move_box, move_side = move
    clone = boxes[:]

    for i, box in enumerate(clone):
        if box[0] == move_box:
            new_sides = box[1].replace(move_side, '')
            clone[i] = (box[0], new_sides) if new_sides != '' else None
            break

    nonone = list(filter(lambda b: not (b is None), clone))

    return nonone

In [ ]:
def is_candidate_closed_by(box, move):
    move_box, move_side = move.split(' ')

    opposites = {
        'T': 'B',
        'B': 'T',
        'L': 'R',
        'R': 'L'
    }

    if box[0] == move_box and box[1] == move_side: # closes box
        return True
    if box[0] != move_box and box[1] == opposites[move_side] and move_box[0] == box[0][0]: # closes same cloumn
        return True
    elif box[0] != move_box and box[1] == opposites[move_side] and move_box[1] == box[0][1]: # closes same row
        return True

    return False

In [ ]:
def closed_boxes(boxes, move):
    candidates = list(filter(lambda b: len(b[1]) == 1, boxes))
    closed = list(filter(lambda b: is_candidate_closed_by(b, move), candidates))

    return len(closed)

In [ ]:
# boxes [('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')] - primera optimizacion, eliminar jugadas repetidas
def minimax(boxes, max_turn, max_score, min_score):
    if len(boxes) == 0: # segunda optimizacion, cortar en profundidad y evaluar
        return max_score - min_score

    if max_turn:
        max_eval = -1000
        children =  flatten_boxes(boxes)
        for move in children:
            new_moves = remove_move(boxes, move)
            new_max_score = max_score + closed_boxes(boxes, move) # tercera optimizacion, usar numero de cajas cerradas para ordenar movimientos
            rating = minimax(new_moves, not max_turn, new_max_score, min_score)
            max_eval = max(rating, max_eval) # cuarta optimizacion, usar poda alfa-beta
        return max_eval
    else:
        min_eval = 1000
        children =  flatten_boxes(boxes)
        for move in children:
            new_moves = remove_move(boxes, move)
            new_min_score = min_score + closed_boxes(boxes, move)
            rating = minimax(new_moves, not max_turn, max_score, new_min_score)
            min_eval = min(rating, min_eval)
        return min_eval


In [ ]:
import datetime

start = datetime.datetime.now()

minimax([('A1', 'LTRB'), ('A2', 'LTRB')], True, 0, 0)

end = datetime.datetime.now()

print((end-start).total_seconds()*1000)

0.10900000000000001
